In [2]:
import pandas as pd
import json

In [ ]:
df1 = pd.read_json('../logs/eval2.log', lines=True)
df2 = pd.read_json('../logs/eval2_model_2.log', lines=True)

# df['error_type'] = df['result'].apply(lambda x: x['error_type'])
# df['error'] = df['result'].apply(lambda x: x['error'])
# df['coverage'] = df['result'].apply(lambda x: x['coverage'])
# df['root_passed'] = df['result'].apply(lambda x: x['test_results']['root_passed'])
# df['root_failed'] = df['result'].apply(lambda x: x['test_results']['root_failed'])
# df['passed'] = df['result'].apply(lambda x: x['test_results']['passed'])
# df['failed'] = df['result'].apply(lambda x: x['test_results']['failed'])
# df['all_passed'] = df['result'].apply(lambda x: x['test_results']['all_passed'])

for _, row in df1.head(10).iterrows():
    print(row['completion'])
    print(row['result']['error'])


				case lines <- newLine:
					scanned++
				}
			}
			if err := scanner.Err(); err != nil {
				if pdebug.Enabled {
					pdebug.Printf("Source: error reading: %v", err)
				}
				return
			}
		}()

		go func() {
			for line := range lines {
				select {
				case <-ctx.Done():
					return
				case refresh <- struct{}{}:
					select {
					case <-ctx.Done():
						return
					case <-refresh:
						continue
					}
				}
				select {
				case <-ctx.Done():
					return
				case s.ChanOutput <- line:
					continue
				case <-refresh:
					continue
				}
			}
		}()

		select {
		case <-ctx.Done():
			return
		case <-s.ready:
			return
		}
	}())
}

// Reset is a no-op.
func (s *Source) Reset() {
}

// Close closes the source.
func (s *Source) Close() {
	s.inClosed = true
}

// Ready returns a channel that will be closed when the source is ready to consume.
func (s *Source) Ready() <-chan struct{} {
	return s.ready
}

// ChanOutput returns a channel that the caller can use to get the output.

In [8]:
print('1', df1['reward'].mean())
print('2', df2['reward'].mean())

row = df1.query('project_path == "35ffa2ac421130af2b8578464a6657aae98295ed/" and relative_file_path == "internal/stringutil/parse.go"').iloc[0].to_dict()

print(row['completion'])

# "project_path": "35ffa2ac421130af2b8578464a6657aae98295ed/", "relative_package_path": "internal/stringutil/", "relative_file_path": "internal/stringutil/parse.go"

1 0.0873136
2 0.07895880000000001
Here is a simple test for the `MatchCaptureGroups` function:

```go
package stringutil

import (
	"regexp"
	"testing"
)

func TestMatchCaptureGroups(t *testing.T) {
	regEx, _ := regexp.Compile("(foo)")
	str := "foobar"
	expected := map[string]string{
		"foo": "foo",
	}
	result := MatchCaptureGroups(regEx, str)

	if len(result) != len(expected) {
		t.Errorf("Expected %v, but got %v", expected, result)
	}

	for key, value := range expected {
		if result[key] != value {
			t.Errorf("Expected %v for key %v, but got %v", value, key, result[key])
		}
	}
}
```

This test checks if the function correctly identifies the named capture groups in the string. It compares the expected result with the actual result, and if they are not equal, it reports an error.

Please note that the error message is just a simple error message, and you can customize it according to your needs.

Also, please note that the `regexp.Compile` function returns a `*Regexp`, which is a poi

In [23]:
df1['root_passed'] = df1.apply(lambda x: x['result']['test_results']['root_passed'], axis=1)
df1['root_failed'] = df1.apply(lambda x: x['result']['test_results']['root_failed'], axis=1)
df1['has_working_tests'] = df1.apply(lambda x: x['root_passed']+x['root_failed'] > 0, axis=1)

print(len(df1.query('has_working_tests == True')))

df2['root_passed'] = df2.apply(lambda x: x['result']['test_results']['root_passed'], axis=1)
df2['root_failed'] = df2.apply(lambda x: x['result']['test_results']['root_failed'], axis=1)
df2['has_working_tests'] = df2.apply(lambda x: x['root_passed']+x['root_failed'] > 0, axis=1)

print(len(df2.query('has_working_tests == True')))

for i in range(len(df1)):
    row1, row2 = df1.iloc[i], df2.iloc[i]

    print(row1['has_working_tests'], row2['has_working_tests'])

    if row1['has_working_tests'] and not row2['has_working_tests']:
        print('worse result', row1['completion'], row1['root_passed']+row1['root_failed'], row1['result'], row2['completion'], row2['root_passed']+row2['root_failed'], row2['result'], sep="\n\n")

    if not row1['has_working_tests'] and row2['has_working_tests']:
        print('better result', row1['completion'], row1['root_passed']+row1['root_failed'], row1['result'], row2['completion'], row2['root_passed']+row2['root_failed'], row2['result'], sep="\n\n")

67
61
False False
True False
worse result

Sure, here's a simple test for the `withDetail` struct. This test will check if the `ErrorDetail` method returns the expected detail, and if the `Error` method returns the original error message.

```go
package hintdetail

import (
	"testing"

	"github.com/stretchr/testify/assert"
)

func TestWithDetail(t *testing.T) {
	err := &withDetail{cause: fmt.Errorf("original error"), detail: "this is a detail"}

	assert.Equal(t, "this is a detail", err.ErrorDetail())
	assert.Equal(t, "original error", err.Error())
	assert.Equal(t, "original error", err.Cause().Error())
	assert.Equal(t, "original error", err.Unwrap().Error())
}
```

This test covers all the methods that you mentioned in your code. You can add more tests to cover other methods as well.

Please note that this is a simple test and does not cover all possible test cases. You should add more tests to cover all possible inputs and their respective outputs.


1

{'error': '', 'error_type': '',

In [ ]:
df1 = pd.read_json('../logs/eval2_no_q.log', lines=True)
df2 = pd.read_json('../logs/eval2_model_2_no_q.log', lines=True)

df1 = df1.head(len(df2))

# df['error_type'] = df['result'].apply(lambda x: x['error_type'])
# df['error'] = df['result'].apply(lambda x: x['error'])
# df['coverage'] = df['result'].apply(lambda x: x['coverage'])
# df['root_passed'] = df['result'].apply(lambda x: x['test_results']['root_passed'])
# df['root_failed'] = df['result'].apply(lambda x: x['test_results']['root_failed'])
# df['passed'] = df['result'].apply(lambda x: x['test_results']['passed'])
# df['failed'] = df['result'].apply(lambda x: x['test_results']['failed'])
# df['all_passed'] = df['result'].apply(lambda x: x['test_results']['all_passed'])

print(len(df1))

for _, row in df1.head(10).iterrows():
    print(row['completion'])
    print(row['result']['error'])

500

				default:
					scanned++
					lines <- newLine
				}
			}
		}()

		select {
		case <-ctx.Done():
			if pdebug.Enabled {
				pdebug.Printf("Bailing out of source setup text reader loop, because ctx was canceled")
			}
			return
		case <-refresh:
			if pdebug.Enabled {
				pdebug.Printf("Refreshing source")
			}
			notifycb()
		case <-s.ready:
			if pdebug.Enabled {
				pdebug.Printf("Source is ready")
			}
		case <-s.setupDone:
			if pdebug.Enabled {
				pdebug.Printf("Source setup is done")
			}
		}
	}())
}

// Reset closes the input reader and resets the state.
func (s *Source) Reset() {
	if s.inClosed {
		return
	}
	s.inClosed = true
	if closer, ok := s.in.(io.Closer); ok {
		closer.Close()
	}
}

// Ready returns a channel that will be closed when the source is ready to
// produce output.
func (s *Source) Ready() <-chan struct{} {
	return s.ready
}

// ChanOutput returns the channel that the source will produce output on.
func (s *Source) ChanOutput() pipeline.ChanOutput {
	r

In [28]:
print('baseline reward: ', df1['reward'].mean())
print('trained reward', df2['reward'].mean())
print('trained/baseline reward', df2['reward'].mean()/df1['reward'].mean())

baseline reward:  0.080252
trained reward 0.08819640000000001
trained/baseline reward 1.0989931715097443


In [29]:
df1['root_passed'] = df1.apply(lambda x: x['result']['test_results']['root_passed'], axis=1)
df1['root_failed'] = df1.apply(lambda x: x['result']['test_results']['root_failed'], axis=1)
df1['has_working_tests'] = df1.apply(lambda x: x['root_passed']+x['root_failed'] > 0, axis=1)

print(len(df1.query('has_working_tests == True')))

df2['root_passed'] = df2.apply(lambda x: x['result']['test_results']['root_passed'], axis=1)
df2['root_failed'] = df2.apply(lambda x: x['result']['test_results']['root_failed'], axis=1)
df2['has_working_tests'] = df2.apply(lambda x: x['root_passed']+x['root_failed'] > 0, axis=1)

print(len(df2.query('has_working_tests == True')))

for i in range(len(df1)):
    row1, row2 = df1.iloc[i], df2.iloc[i]

    print(row1['has_working_tests'], row2['has_working_tests'])

    if row1['has_working_tests'] and not row2['has_working_tests']:
        print('worse result', row1['completion'], row1['root_passed']+row1['root_failed'], row1['result'], row2['completion'], row2['root_passed']+row2['root_failed'], row2['result'], sep="\n\n")

    if not row1['has_working_tests'] and row2['has_working_tests']:
        print('better result', row1['completion'], row1['root_passed']+row1['root_failed'], row1['result'], row2['completion'], row2['root_passed']+row2['root_failed'], row2['result'], sep="\n\n")

64
70
False False
False False
False False
False False
False False
False False
False False
False True
better result

Here's a sample test for the `loadFSHandler` function. This test uses the `os` and `os/exec` packages to simulate the `os.Stat` and `os.Open` functions.

```go
package auth

import (
	"io/ioutil"
	"net/http"
	"os"
	"path/filepath"
	"strings"
	"testing"

	"github.com/stretchr/testify/assert"
)

func TestLoadFSHandler(t *testing.T) {
	// Create a temporary directory
	dir, err := ioutil.TempDir("", "TestLoadFSHandler")
	if err != nil {
		t.Fatal(err)
	}
	defer os.RemoveAll(dir)

	// Create a test file
	testFile := filepath.Join(dir, "test.txt")
	err = ioutil.WriteFile(testFile, []byte("Test file content"), 0644)
	if err != nil {
		t.Fatal(err)
	}

	// Load the filesystem handler
	handler, err := loadFSHandler()
	if err != nil {
		t.Fatal(err)
	}

	// Make a request to the handler
	resp, err := http.Get("http://localhost" + handler.ServeHTTPPrefix + "/test.txt")
	if err != ni

In [13]:
original_model = 'original'
trained_model = 'model_11_550'

original_df = pd.read_json('../logs/'+original_model+'/score.log', lines=True)
trained_df = pd.read_json('../logs/'+trained_model+'/score.log', lines=True)
original_fixed_df = pd.read_json('../logs/'+original_model+'/score_fixed.log', lines=True)
trained_fixed_df = pd.read_json('../logs/'+trained_model+'/score_fixed.log', lines=True)

original_df = original_df.head(len(trained_df))
original_fixed_df = original_fixed_df.head(len(trained_fixed_df))

# df['error_type'] = df['result'].apply(lambda x: x['error_type'])
# df['error'] = df['result'].apply(lambda x: x['error'])
# df['coverage'] = df['result'].apply(lambda x: x['coverage'])
# df['root_passed'] = df['result'].apply(lambda x: x['test_results']['root_passed'])
# df['root_failed'] = df['result'].apply(lambda x: x['test_results']['root_failed'])
# df['passed'] = df['result'].apply(lambda x: x['test_results']['passed'])
# df['failed'] = df['result'].apply(lambda x: x['test_results']['failed'])
# df['all_passed'] = df['result'].apply(lambda x: x['test_results']['all_passed'])

print(len(original_df), len(original_fixed_df))

# for _, row in df1.head(10).iterrows():
#     print(row['completion'])
#     print(row['result']['error'])

500 500


In [ ]:
original_reward = original_df['reward'].mean()
trained_reward = trained_df['reward'].mean()
original_fixed_reward = original_fixed_df['best_reward'].mean()
trained_fixed_reward = trained_fixed_df['best_reward'].mean()

report = [{
    'type': 'no fixing iteration',
    'original reward': '%.4f' % original_reward,
    'trained reward': '%.4f' % trained_reward,
    'trained/original': '%.4f' % (trained_reward/original_reward),
    'uplift, %': '%.2f' % (trained_reward/original_reward*100 - 100),
},
{
    'type': 'with fixing iteration',
    'original reward': '%.4f' % original_fixed_reward,
    'trained reward': '%.4f' % trained_fixed_reward,
    'trained/original': '%.4f' % (trained_fixed_reward/original_fixed_reward),
    'uplift, %': '%.2f' % (trained_fixed_reward/original_fixed_reward*100 - 100),
},
]

report_df = pd.DataFrame(data=report)

report_df.to_markdown('../logs/report.md', index=False)

pd.DataFrame(data=report)

,type,original reward,trained reward,trained/original,"uplift, %"
0,no fixing iteration,0.1731,0.2023,1.1690,16.90
1,with fixing iteration,0.1757,0.2085,1.1870,18.70
